In [92]:
!pip install spacy-wordnet

^C


In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter

In [2]:

os.getcwd()
entertainment=  pd.read_csv('Dataset/Entertainment/Entertainment_Dataset.csv')
entertainment.columns = ['unnamed', 'values', 'category']
entertainment = entertainment.drop('unnamed', axis = 1)

insurance=  pd.read_csv('./Dataset/insurance/insurance_dataset.csv')
insurance.columns = ['unnamed', 'values', 'category']
insurance = insurance.drop('unnamed', axis = 1)

finance=  pd.read_csv('./Dataset/finance/finance_dataset.csv')
finance.columns = ['unnamed', 'values', 'category']
finance = finance.drop('unnamed', axis = 1)

travel = pd.read_csv('./Dataset/travel/Travel_Dataset.csv')
travel.columns = ['unnamed', 'values', 'category']
travel = travel.drop('unnamed', axis = 1)

medical = pd.read_csv('./Dataset/Medical/medical_dataset.csv')
medical.columns = ['unnamed', 'values', 'category']
medical = medical.drop('unnamed', axis = 1)

l = [insurance, entertainment, finance, travel, medical]
df = pd.concat(l)

In [3]:
df.head()
Counter(df['category'])

Counter({'insurance': 1000,
         'entertainment': 386,
         'finance': 44999,
         nan: 10,
         'travel': 4977,
         'medical': 13200})

In [4]:

from textgenie import TextGenie
from tqdm import tqdm
textgenie = TextGenie(
                    "hetpandya/t5-small-tapaco",
                    "bert-base-uncased",
                    "en_core_web_sm",
                    device = 'cuda:0'
                     )

def augmentor(dataset):
    es = list(eval(f"{dataset}['values']"))
    aug = []
    for x in tqdm(range(len(es))):
        es[x] = es[x].replace('\n', ' ')
        es[x] = es[x].replace("\'", ' ')
        try:
            l = textgenie.magic_once(
                                    f"{es[x]}",
                                    "paraphrase: ",
                                    n_paraphrase_predictions=2 ,
                                    n_mask_predictions=1,
                                    convert_to_active = False
                                    )
            
            aug.extend(l)
        except:
            continue
    es.extend(aug)
    return es

Loading Paraphrase Model..
Loading Mask Fill Model..


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# It  takes about 12 hours to run below code on GPU !!

In [5]:
# ent = augmentor('entertainment')
# tra = augmentor('travel')
# ins = augmentor('insurance')

In [6]:
# def to_dataset(listname, category):
#     a = eval(listname)
#     df = pd.DataFrame(zip(a, [category]*len(a)) ,columns = ['values', 'category'])
#     return df
# to_dataset('ent', 'entertainment').to_csv('Dataset/Entertainment/entertainment_aug.csv')
# to_dataset('tra', 'travel').to_csv('Dataset/Travel/travel_aug.csv')
# to_dataset('ins', 'insurance').to_csv('Dataset/Insurance/insurance_aug.csv')

In [7]:
entertainment = pd.read_csv('Dataset/Entertainment/entertainment_aug.csv')
entertainment.columns = ['unnamed', 'values', 'category']
entertainment = entertainment.drop('unnamed', axis = 1)

travel = pd.read_csv('Dataset/Travel/travel_aug.csv')
travel.columns = ['unnamed', 'values', 'category']
travel = travel.drop('unnamed', axis = 1)

insurance = pd.read_csv('Dataset/Insurance/insurance_aug.csv')
insurance.columns = ['unnamed', 'values', 'category']
insurance = insurance.drop('unnamed', axis = 1)

In [8]:
entertainment

,values,category
0,Gallery unveils interactive tree A Christmas ...,entertainment
1,Jarre joins fairytale celebration French musi...,entertainment
2,Musical treatment for Capra film The classic ...,entertainment
3,Richard and Judy choose top books The 10 auth...,entertainment
4,Poppins musical gets flying start The stage a...,entertainment
...,...,...
20872,buffy creator joins wonder woman the creator o...,entertainment
20873,buffy creator joins wonder woman the creator o...,entertainment
20874,buffy creator joins wonder woman the creator o...,entertainment
20875,buffy creator joins wonder woman the creator o...,entertainment


In [9]:
l = [insurance, entertainment, finance, travel, medical]
df = pd.concat(l)

In [10]:
df

,values,category
0,can you borrow against globe Life Insurancebor...,insurance
1,do Medicare cover my spouseif your spouse have...,insurance
2,what happen when you change homeowner insuranc...,insurance
3,what be a typical renter insurance costI be su...,insurance
4,what be car insurance base oncar insurance rat...,insurance
...,...,...
13195,"([""Intact function of the Forkhead Box P2 (FOX...",medical
13196,"([""Studies on ADHD in educational settings ind...",medical
13197,(['The mechanisms underlying cerebellar learni...,medical
13198,(['Withania somnifera root extract has been us...,medical


In [11]:
print(Counter(df['category']))
categories = list(dict(Counter(df['category'])).keys())

Counter({'finance': 44999, 'travel': 38451, 'insurance': 25801, 'entertainment': 20877, 'medical': 13200, nan: 10})


In [12]:
categories

['insurance', 'entertainment', 'finance', nan, 'travel', 'medical']

In [13]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['values'])
print(X.shape)

(143338, 196738)


In [14]:
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X)
print(X.shape)

(143338, 196738)


In [15]:
mp = {categories[x]:x for x in range(len(categories))}

y_true = []
for x in df['category']:
    y_true.append(mp[x])

In [16]:
import numpy as np
y_true = np.array(y_true)
y_true.shape

(143338,)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=0.2, random_state=42)

In [18]:
# Train the model
clf = LogisticRegression(max_iter = 300)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=300)

In [19]:
# Evaluate the model
from sklearn.metrics import balanced_accuracy_score as bas
from sklearn.metrics import roc_auc_score as ras
from sklearn.metrics import confusion_matrix as cm

y_pred = clf.predict(X_test)

accuracy = bas(y_test, y_pred)
print(f"Accuracy: {accuracy}")

conf_m = cm(y_test, y_pred)


y_pred = clf.predict_proba(X_test)
accuracy = ras(y_test, y_pred, multi_class='ovr')
print(f"roc_auc_score: {accuracy}")

print(f"confusion matrix: \n {conf_m}")

Accuracy: 0.8320242993294364
roc_auc_score: 0.9283707493431214
confusion matrix: 
 [[5140    0    6    0    1    1]
 [   0 4285    3    0    0    0]
 [   4    0 8847    0    0    0]
 [   0    0    2    0    0    0]
 [   3    1   15    0 7779    0]
 [   0    0    7    0    0 2574]]


In [20]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X_train, y_train)

BernoulliNB()

In [21]:
y_pred = clf.predict(X_test)

accuracy = bas(y_test, y_pred)
print(f"Accuracy: {accuracy}")

conf_m = cm(y_test, y_pred)


y_pred = clf.predict_proba(X_test)
accuracy = ras(y_test, y_pred, multi_class='ovr')
print(f"roc_auc_score: {accuracy}")

print(f"confusion matrix: \n {conf_m}")

Accuracy: 0.7933499810118777
roc_auc_score: 0.9154886641358875
confusion matrix: 
 [[5143    0    0    0    5    0]
 [  23 4233    1    0   31    0]
 [  58    0 7275    0 1511    7]
 [   0    0    1    0    1    0]
 [   0    0    0    0 7798    0]
 [  75    1   32    0   16 2457]]


In [39]:
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV
clf = CalibratedClassifierCV(svm.LinearSVC())
clf.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=LinearSVC())

In [40]:
y_pred = clf.predict(X_test)

accuracy = bas(y_test, y_pred)
print(f"Accuracy: {accuracy}")

conf_m = cm(y_test, y_pred)


y_pred = clf.predict_proba(X_test)
accuracy = ras(y_test, y_pred, multi_class='ovr')
print(f"roc_auc_score: {accuracy}")

print(f"confusion matrix: \n {conf_m}")

Accuracy: 0.9158080072141971
roc_auc_score: 0.9998033333674085
confusion matrix: 
 [[5145    0    1    0    1    1]
 [   0 4288    0    0    0    0]
 [   3    0 8847    0    0    1]
 [   0    0    1    1    0    0]
 [   4    1   12    0 7781    0]
 [   0    0    5    0    0 2576]]


In [36]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = CalibratedClassifierCV(make_pipeline(StandardScaler(with_mean=False),
                    SGDClassifier(max_iter=1000, tol=1e-3)))
clf.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=Pipeline(steps=[('standardscaler',
                                                       StandardScaler(with_mean=False)),
                                                      ('sgdclassifier',
                                                       SGDClassifier())]))

In [37]:
y_pred = clf.predict(X_test)

accuracy = bas(y_test, y_pred)
print(f"Accuracy: {accuracy}")

conf_m = cm(y_test, y_pred)

y_pred = clf.predict_proba(X_test)
accuracy = ras(y_test, y_pred, multi_class='ovr')
print(f"roc_auc_score: {accuracy}")

print(f"confusion matrix: \n {conf_m}")

Accuracy: 0.7700024829213129
roc_auc_score: 0.9497226364138838
confusion matrix: 
 [[5108    2   30    0    8    0]
 [   1 4223   55    0    9    0]
 [   0    0 7731    0 1120    0]
 [   0    0    1    0    1    0]
 [   6    3    6    0 7783    0]
 [   2    3  582    0    3 1991]]


# Best accuracy is given by the LinearSVM classifier.

In [44]:
new_data = [
            'flights from mumbai to japan',
           '5000 rupees dollar to be transferred tommorow',
           'health insurance is realy expensive',
           'the pericardium is swollen',
           'the Lord Of The Rings is a great movie',
           'liver surgery is needed',
           'The largest organ on the body is the skin also known as derma',
            'the train is late again',
            'let us get on the bus',
            'we can go to the train station using the car'
           ]
X_new = vectorizer.transform(new_data)
X_new = tfidf_transformer.transform(X_new)
y_new = clf.predict(X_new)
for x in y_new:
    print(categories[x])

travel
finance
insurance
finance
entertainment
medical
finance
travel
travel
travel
